# Import the necessary libraries and modules.

In [4]:
import numpy as np
import GPyOpt
import pandas as pd
from ml_models import OptimizationModel

# Update experiment_data DataFrame after each batch.

In [5]:
def _update_experiment_data(self, wellnames, recipes, Experiment_result, GP_Prediction):
    new_data = pd.DataFrame({
        'Recipes': recipes,
        'WellNames': wellnames, 
        'Experiment Result': Experiment_result,
        'GP Prediction': GP_Prediction
    })
    self.experiment_data = pd.concat([self.experiment_data, new_data], ignore_index=True)

# Dataframe we will be updating.

In [6]:
experiment_data = pd.DataFrame(columns = ["Recipes", "Wellnames", "Experiment_result", "GP_Prediction"])

# Define the Branin function

Branin function (look up graph + equation as reference), has three global minimas. We are using it as a way to test our optimization model as we're trying to exploit/explore these global minimas using our acquisition function.

Branin function will behave as our objective function here. Hopefully, our GP model will learn the space of the branin function by sampling + evaluating different points (acquisition function will balance exploitation vs exploration).
Hopefully it will find global minimas.

In [1]:
def branin(x):
    # The recommended values of the parameters
    a = 1
    b = 5.1 / (4 * np.pi**2)
    c = 5 / np.pi
    r = 6
    s = 10
    t = 1 / (8 * np.pi)
    # The expression of the function
    return (a * (x[:, 1] - b * x[:, 0]**2 + c * x[:, 0] - r)**2 + s * (1 - t) * np.cos(x[:, 0]) + s).reshape(-1, 1)

# Training loop.

In [ ]:
import random

def generate_shaha(index):
    return f"shaha{index}{random.randint(0, 9999):04d}"

In [ ]:
# hardcode batch_size = 5
# hardcode y_shape to be 2 assuming we are fixing 3 reagents and varing 2 of em.
batch_size = 5
y_shape = 2
def run(self, model):
        # define variables.
        self.batch_num = 0 
        recipes = np.ones((batch_size, y_shape)) * 3.1415e-2
        
        # RUN FIRST RECIPE.
        print('<<controller>> executing batch {}'.format(self.batch_num))
        
        # generate new wellnames for next batch
        wellnames = [self.generate_shaha(i) for i in range(recipes.shape[0])]
        
        # Use the Branin function to generate "scan data"
        scan_data = branin(recipes)

        model.train(recipes, scan_data)
                
        self.batch_num += 1

        # ENTER LOOP TO TRAIN RECIPES.
        while not model.quit:
            model.train(recipes, scan_data)
            print('<<controller>> executing batch {}'.format(self.batch_num))
            
            # generate new wellnames for next batch
            wellnames = [self.generate_shaha(i) for i in range(recipes.shape[0])]
            
            # TODO: last_filename gets in data from the create_samples sheet. Use our "function" to pull in fake data/replicate create_samples.
            
            # pull in wavelength light absorption level.
            scan_data = ???
            
            # generate the predictions for the next round
            gp_prediction = model.predict()
            self.batch_num += 1
            
            # update our experiment data TODO.
            self._update_experiment_data(wellnames, recipes, scan_data, gp_prediction)
            recipes = gp_prediction
            
        return
    